In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from model import PINN

: 

: 

In [ ]:
# Define callbacks
def epoch_logger(epoch, model, physics_loss, data_loss, **kw):
    if epoch % 1000 != 0:
        return
    print(
        "Epoch %d: Physics Loss %.4g, Node3YVel Loss %.4g, Node4YVel Loss %.4g, E = %.4g"
        % (epoch, physics_loss, data_loss[0], data_loss[1], model.E())
    )


# Define model
layers = [1] + 2 * [16] + [4]
sigmas = [1, 20, 50]
model = PINN(layers, sigmas)
model.load_ops_data()
model.compile(
    torch.optim.Adam(list(model.parameters()) + [model.a], lr=1e-4),
    callbacks=[epoch_logger],
    loss_weights=[1e-10, 1, 1],
)
model.train()

True
Epoch 0: Physics Loss 9.822e+04, Node3YVel Loss 48.06, Node4YVel Loss 21.87, E = 0.3733


KeyboardInterrupt: 